In [ ]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,188550.56,-2.95,-1.79,-1.20,-0.16,-0.19,0.08,-0.09,-0.00
3,-2.95,42550.62,-0.61,0.15,0.08,-0.01,-0.01,0.01,0.02
4,-1.79,-0.61,10773.53,0.05,0.09,-0.01,-0.01,-0.00,-0.00
5,-1.20,0.15,0.05,1802.56,-0.01,-0.00,0.00,-0.01,-0.00
6,-0.16,0.08,0.09,-0.01,391.11,0.01,0.00,0.00,0.00
7,-0.19,-0.01,-0.01,-0.00,0.01,98.69,0.00,0.00,-0.00
8,0.08,-0.01,-0.01,0.00,0.00,0.00,33.29,0.00,0.00
9,-0.09,0.01,-0.00,-0.01,0.00,0.00,0.00,11.23,-0.00
10,-0.00,0.02,-0.00,-0.00,0.00,-0.00,0.00,-0.00,1.88


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00007,-0.00002,-0.00005,0.00003,-0.00006,-0.00000
3,-0.00003,1.00000,-0.00003,0.00002,0.00002,-0.00000,-0.00001,0.00002,0.00006
4,-0.00004,-0.00003,1.00000,0.00001,0.00004,-0.00001,-0.00002,-0.00000,-0.00001
5,-0.00007,0.00002,0.00001,1.00000,-0.00001,-0.00000,0.00000,-0.00005,-0.00005
6,-0.00002,0.00002,0.00004,-0.00001,1.00000,0.00004,0.00002,0.00002,0.00001
7,-0.00005,-0.00000,-0.00001,-0.00000,0.00004,1.00000,0.00002,0.00007,-0.00000
8,0.00003,-0.00001,-0.00002,0.00000,0.00002,0.00002,1.00000,0.00002,0.00000
9,-0.00006,0.00002,-0.00000,-0.00005,0.00002,0.00007,0.00002,1.00000,-0.00004
10,-0.00000,0.00006,-0.00001,-0.00005,0.00001,-0.00000,0.00000,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

5.265841524390419

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.95369786e+09 1.98150879e+08 1.24282383e+07 3.38892618e+05
 1.53389749e+04 9.23467603e+02 9.40673012e+01 7.85085309e+00
 7.79859448e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999533,-0.029688,-0.007117,-0.001192,-0.000258,-0.000066,-0.000021,-0.000008,-0.000001
1,0.029382,0.998795,-0.038811,-0.006024,-0.001304,-0.000330,-0.000111,-0.000037,-0.000006
2,0.008216,0.038386,0.998791,-0.028911,-0.006070,-0.001533,-0.000518,-0.000174,-0.000029
3,0.001588,0.007012,0.028323,0.998466,-0.045578,-0.010891,-0.003661,-0.001239,-0.000208
4,0.000410,0.001810,0.007154,0.044450,0.996465,-0.067316,-0.021096,-0.007058,-0.001179
5,0.000126,0.000554,0.002189,0.013111,0.063324,0.989360,-0.124967,-0.036433,-0.005998
6,0.000055,0.000245,0.000968,0.005777,0.026748,0.114336,0.975025,-0.186604,-0.026111
7,0.000029,0.000128,0.000507,0.003030,0.013944,0.055595,0.173675,0.967815,-0.172849
8,0.000010,0.000043,0.000172,0.001028,0.004727,0.018736,0.055559,0.164725,0.984583


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00046686, 0.00120457, 0.00120891, 0.0015336 , 0.0035353 ,
       0.01064014, 0.02497457, 0.0321851 , 0.01541674])